In [1]:
!pip install pennylane pennylane-qiskit tensorflow qiskit

  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.0 MB ? eta -:--:--
   ---------------------------------------- 2.1/390.0 MB 10.7 MB/s eta 0:00:37
   ---------------------------------------- 2.9/390.0 MB 11.2 MB/s eta 0:00:35
   ---------------------------------------- 3.1/390.0 MB 5.8 MB/s eta 0:01:08
    --------------------------------------- 7.9/390.0 MB 9.2 MB/s eta 0:00:42
   - -------------------------------------- 10.0/390.0 MB 9.4 MB/s eta 0:00:41
   - -------------------------------------- 11.8/390.0 MB 9.2 MB/s eta 0:00:41
   - -------------------------------------- 14.2/390.0 MB 9.6 MB/s eta 0:00:40
   - -------------------------------------- 16.5/390.0 MB 9.7 MB/s eta 0:00:39
   - -----------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import tensorflow as tf
import warnings

# Set TensorFlow logging level to ERROR
tf.get_logger().setLevel('ERROR')

# Suppress specific warnings from TensorFlow
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')

# Load and preprocess the dataset
data = pd.read_csv('C:\\Users\\Nirusan03\\PycharmProjects\\FYP_POC\\Final_Dataset.csv')

# Select features and target column
X = data.drop(columns=['Label', 'Label.1'])
y = data['Label']

# Encode labels to binary format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA for dimensionality reduction, retaining 20 components
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [2]:
import pennylane as qml
import tensorflow as tf

# Set up a quantum device with 20 qubits
n_qubits = 20
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum convolutional layer
def qconv_layer(x):
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))
    for i in range(n_qubits):
        qml.RY(0.25 * 3.14159, wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

# Quantum node ensuring only real-valued outputs
@qml.qnode(dev, interface="tf")
def quantum_circuit(inputs):
    qconv_layer(inputs)
    # Expectation values are real, so we avoid complex outputs entirely
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define QuantumLayer without complex casting warnings
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        # Initialize a trainable weight for each qubit using TensorFlow
        self.weight = tf.Variable(tf.random.normal([n_qubits]), trainable=True)

    def call(self, inputs):
        # Scale inputs to match the number of qubits
        scaled_inputs = inputs * tf.reshape(self.weight, (1, -1))

        # Apply quantum circuit to each input sample in the batch
        quantum_outputs = tf.vectorized_map(quantum_circuit, scaled_inputs)

        # Ensure outputs are float32 and real-valued
        quantum_outputs_real = tf.math.real(quantum_outputs)
        
        return tf.cast(quantum_outputs_real, dtype=tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], n_qubits)


In [3]:
# Define the hybrid quantum-classical model
model = tf.keras.models.Sequential([
    QuantumLayer(),                             # Quantum feature extraction layer
    tf.keras.layers.Dense(128, activation='relu', input_shape=(20,)),  # Larger dense layer for more capacity
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile with a binary cross-entropy loss function, which is better suited for classification
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Nirusan03\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
tf.config.run_functions_eagerly(True)

In [5]:
# Train the model
history = model.fit(
    X_train_pca, 
    y_train, 
    epochs=50,
    batch_size=32,
    validation_data=(X_test_pca, y_test)
)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_pca, y_test)
print(f"Test accuracy: {accuracy:.4f}")

Epoch 1/50


: 